# RNA MULTICAPA DE 3 CAPAS
 - capa de entrada(N)
 - 1 capa oculta(M)
 - capa de salida(O)

## importar librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

## funcion de activacion

In [ ]:
def func_activacion(n):
  return 1/(1+math.exp(-n))   

In [ ]:
func_activacion_vect = np.vectorize(func_activacion)

## RNA

In [ ]:
def neurona(pesos,entrada, bias):
  prod_punto = np.dot(pesos,entrada)+bias # z = W.X + b
  return func_activacion_vect(prod_punto) # f(z)

## RNA Multicapa

In [ ]:
def red_neuronal_multicapa(input, pesosc1, biasc1, pesoscs, biascs):
  # Capa entrada 
  entrada_x = input
  # Capa oculta 1
  salida_c1 = neurona(pesosc1, entrada_x, biasc1)
  # Capa de SALIDA
  output_cs = neurona(pesoscs, salida_c1, biascs)
  # Retornar resultados  
  return salida_c1, output_cs

## derivada de la funcion

In [ ]:
def derivada_f(salida_y): # derivada de la funcion sigmoide
  return salida_y*(1-salida_y)

## backpropagation

In [ ]:
def propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error=0.005,epochs=1000):
  # longitud de la entrada(caracteristicas)
  n_entradas = entradas.shape[1]
  # Capa oculta 1
  pesosc1 = 2*np.random.rand(neuronas_c1,n_entradas) -1   # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
  biasc1 = 2*np.random.rand(neuronas_c1)-1
  # Capa de salida
  pesoscs = 2*np.random.rand(neuronas_cs,neuronas_c1) - 1 
  biascs = 2*np.random.rand(neuronas_cs) - 1
  # Iterar el algoritmo(nro de epocas)
  for epoch in range(0,epochs):
    for entrada_i,salida_deseada_i in zip(entradas,salidas_deseadas):
      salida_c1, output_cs = red_neuronal_multicapa(entrada_i, pesosc1, biasc1, pesoscs, biascs)
      # ----------------------------------------------------------------------------------------------------
      # Calcular los errores producidos en cada capa de forma invertida(retropropagacion) y actualizar pesos
      # ----------------------------------------------------------------------------------------------------
      SD = np.zeros(neuronas_cs) # vector con el nro de neuronas
      SD[salida_deseada_i - 1] = 1
      # Calcular Δ para la CAPA de SALIDA(cs): 
      delta_cs = derivada_f(output_cs)*(SD - output_cs)
      peso_anterior_cs = pesoscs
      # Actualizar pesos que llegan a la capa de SALIDA
      pesoscs = pesoscs + alfa * delta_cs.reshape(1,-1).T * salida_c1.reshape(1,-1) 
      biascs = biascs + alfa * delta_cs*1   
      # Calcular Δ para la CAPA OCULTA 1 (pesosc1):      
      delta_c1 = derivada_f(salida_c1) * np.dot(peso_anterior_cs.T, delta_cs) 
      # Actualizar pesos  que entran a las neuronas de la capa oculta 1
      pesosc1 = pesosc1 + alfa * (delta_c1.reshape(1,-1)*entrada_i.reshape(1,-1).T).T 
      biasc1 = biasc1 + alfa * delta_c1 * 1  

  return pesosc1,biasc1,pesoscs,biascs 

## fit RNA

In [ ]:
def fit_rna(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs):
  pesosc1,biasc1,pesoscs,biascs,loss = propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs)
  return pesosc1,biasc1,pesoscs,biascs,loss

## predict RNA

In [ ]:
def predict_rna(x_test, valores_obtenidos):
  pesosc1,biasc1,pesoscs,biascs = valores_obtenidos[0],valores_obtenidos[1],valores_obtenidos[2],valores_obtenidos[3]
  y_predict_list = []
  for entrada in x_test:
    salida_c1, output_cs = red_neuronal_multicapa(entrada, pesosc1,biasc1,pesoscs,biascs)
    y_predict_list.append(output_cs)
  return y_predict_list

# DETECCION DE DIABETES CON RNA

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv("wine.csv")
df.head()

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280_OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# =========== NORMALIZANDO =========== 
X = df.drop("class", axis=1)
Y = df["class"]

scaler = preprocessing.StandardScaler()
scaler.fit(X)
# =========== TRANSFORMANDO =========== 
X = scaler.transform(X)

In [ ]:
# =========== SEPARACION DE DATOS =========== 
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

## modelamiento

In [ ]:
# =========== USANDO RNA MULTICAPA DE 3 CAPAS =========== 
pesosc1,biasc1,pesoscs,biascs,loss = fit_rna(X_train,y_train,0.001,12,3,0.005,100)

In [ ]:
valores_train = [pesosc1,biasc1,pesoscs,biascs] # recuperar los pesos y bias optimos
y_pred = predict_rna(X_test, valores_train)     # predecir

In [ ]:
valores_pred = []
for vector in y_pred:
  indice = np.argmax(vector)
  valores_pred.append(indice+1)

In [ ]:
y_pred = pd.Series(valores_pred)

In [ ]:
print(classification_report(y_test,y_pred ))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        14
           2       1.00      0.93      0.96        14
           3       0.89      1.00      0.94         8

    accuracy                           0.97        36
   macro avg       0.96      0.98      0.97        36
weighted avg       0.98      0.97      0.97        36

